In [1]:
from functools import partial
from importlib import reload
import unittest

In [2]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run cell',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.restart_run_all();
        return false;
    }}
);

<IPython.core.display.Javascript object>

In [3]:
def print_function(*args, **kwargs):
    print('args={args}, kwargs={kwargs}'.format(args=args, kwargs=kwargs))

class AddChannelFunctions:
    def __init__(self, channels, functions):
        self.channels = channels
        self.functions = functions
    
    def __call__(self, cls):
        def print_function(*args, **kwargs):
            print('args={args}, kwargs={kwargs}'.format(args=args, kwargs=kwargs))
            
        for channel in self.channels:
            for function in self.functions:
                print_function_targeted = partial(print_function, ch=channel, function=function)
                exec("cls.{ch}_{fn} = print_function_targeted".format(ch=str(channel), fn=function))
#         cls.ch1_trig_in = print_function
        return cls

# Test PulseSequence

In [11]:
from silq.pulses import PulseSequence, DCPulse, TriggerPulse, SinePulse

class TestPulseSequence(unittest.TestCase):
    def setUp(self):
        self.pulse_sequence = PulseSequence()
        
    def test_add_remove_pulse(self):
        pulse = DCPulse(name='dc', amplitude=1.5, duration=10, t_start=0)
        self.pulse_sequence.add(pulse)
        self.assertIn(pulse, self.pulse_sequence)

        # Remove pulses
        self.pulse_sequence.clear()
        self.assertEqual(len(self.pulse_sequence.pulses), 0)
        
    def test_sort(self):
        pulse1 = DCPulse(name='dc1', amplitude=1.5, duration=10, t_start=1)
        pulse2 = DCPulse(name='dc2', amplitude=1.5, duration=10, t_start=0)
        self.pulse_sequence.add(pulse1)
        self.pulse_sequence.add(pulse2)
        self.assertEqual(pulse2, self.pulse_sequence[0])

In [12]:
test_pulse_sequence = TestPulseSequence()

suite = unittest.TestLoader().loadTestsFromModule(test_pulse_sequence)
unittest.TextTestRunner().run(suite)

..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

# Test ArbStudio

In [13]:
channels = ['ch1', 'ch2', 'ch3', 'ch4']
functions = ['trigger_source', 'trigger_mode', 'add_waveform', 'sequence']

@AddChannelFunctions(channels, functions)
class MockArbStudio:
    def __init__(self):
        pass
    
arbstudio = MockArbStudio()

In [19]:
from silq.meta_instruments.instrument_interfaces import get_instrument_interface

class TestArbStudio(unittest.TestCase):
    def setUp(self):
        self.pulse_sequence = PulseSequence()
        self.arbstudio = MockArbStudio()
        self.arbstudio_interface = get_instrument_interface(self.arbstudio)
        
    def test_pulse_implementation(self):
        sine_pulse = SinePulse(t_start=0, duration=10, frequency=1e6, amplitude=1)
        self.assertIsNone(self.arbstudio_interface.get_pulse_implementation(sine_pulse))
        
        DC_pulse = DCPulse(t_start=0, duration=10, amplitude=1)
        self.assertIsNotNone(self.arbstudio_interface.get_pulse_implementation(DC_pulse))
        DC_pulse.amplitude=3
        self.assertIsNone(self.arbstudio_interface.get_pulse_implementation(DC_pulse))
        
    def test_ELR_programming(self):
        empty_pulse = DCPulse(name='empty', t_start=0, duration=10, amplitude=1.5)
        load_pulse = DCPulse(name='load', t_start=10, duration=10, amplitude=-1.5)
        read_pulse = DCPulse(name='read', t_start=20, duration=10, amplitude=0)
        pulses = [empty_pulse, load_pulse, read_pulse]
        for pulse in pulses:
            self.arbstudio_interface.pulse_sequence.add(pulse)
        self.arbstudio_interface.setup()

In [20]:
test_arbstudio = TestArbStudio()

suite = unittest.TestLoader().loadTestsFromModule(test_arbstudio)
unittest.TextTestRunner().run(suite)

E.
ERROR: test_ELR_programming (__main__.TestArbStudio)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-19-c9392912dd3b>", line 25, in test_ELR_programming
    self.arbstudio_interface.setup()
  File "C:\Users\serwa_000\Documents\GitHub\SilQ\silq\meta_instruments\instrument_interfaces\lecroy\ArbStudio1104.py", line 37, in setup
    for pulse in self.pulse_sequence])
  File "C:\Users\serwa_000\Documents\GitHub\SilQ\silq\meta_instruments\instrument_interfaces\lecroy\ArbStudio1104.py", line 37, in <listcomp>
    for pulse in self.pulse_sequence])
AttributeError: 'NoneType' object has no attribute 'output_channel'

----------------------------------------------------------------------
Ran 2 tests in 0.001s

FAILED (errors=1)


<unittest.runner.TextTestResult run=2 errors=1 failures=0>

# Other

In [10]:
from silq.meta_instruments.PulseSequence import PulseSequence
from silq.meta_instruments import pulses

ImportError: No module named 'silq.meta_instruments.PulseSequence'

In [ ]:
pulse_sequence = PulseSequence()

In [ ]:
sine_pulse1 = pulses.SinePulse(frequency=1e6, amplitude=1, t_start=0, t_stop=2)
sine_pulse2 = pulses.SinePulse(frequency=.4e6, amplitude=1, t_start=1, t_stop=3)

In [ ]:
pulse_sequence.add_pulse(sine_pulse2)
pulse_sequence.add_pulse(sine_pulse1)
pulse_sequence.sort_pulses()

In [ ]:
x = [[],[],[]]

In [ ]:
x[0] += [2]

In [ ]:
x.sort()

In [ ]:
class TestClass:
    def __init__(self, pulses):
        self.pulses = pulses
        
    def __getitem__(self, index):
        return self.pulses[index]
    
    def fun1(self):
        print('fun1')
        
    @staticmethod
    def fun2():
        print('fun2')

In [ ]:
class TestClassChild(TestClass):
    def __init__(self, pulses):
        self.pulses = pulses
        
    def __getitem__(self, index):
        return self.pulses[index]
    
    def fun1(self):
        print('fun1')
        
    @staticmethod
    def fun2():
        print('fun2')

In [ ]:
test_class = TestClassChild([1,2,3])

In [ ]:
isinstance(test_class, TestClass)